In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

dfTrain = pd.read_csv("./NewTrainSet")
dfTrain = dfTrain.drop(columns=['DMA', 'Unnamed: 0'])

dfTest = pd.read_csv("./NewTestSet")
dfTest = dfTest.drop(columns=['DMA', 'Unnamed: 0'])

In [7]:
def df_to_numpy_array(df):
    num_columns = list(df.select_dtypes(include=['float64', 'int64']).columns.values)
    num_values = df[num_columns].values
    bool_columns = list(df.select_dtypes(include=['bool']).columns.values)
    bool_values = df[bool_columns].values
    return np.concatenate((bool_values,num_values),axis=1)

In [8]:
XTrain = df_to_numpy_array(dfTrain.drop(columns=["LeakFixIn4Days", "t20Leak", "t24Leak", "t28Leak", "t32Leak", "t36Leak", "t40Leak"]))
XTest = df_to_numpy_array(dfTest.drop(columns=["LeakFixIn4Days", "t20Leak", "t24Leak", "t28Leak", "t32Leak", "t36Leak", "t40Leak"]))
yTrain = df_to_numpy_array(dfTrain.drop(columns=["Flow","dev","t-4dev","t-8dev","t-12dev","t-16dev","t-20dev","t-24dev","t-28dev","t-32dev","t-36dev","t-40dev","t20Leak", "t24Leak", "t28Leak", "t32Leak", "t36Leak", "t40Leak"]))
yTest = df_to_numpy_array(dfTest.drop(columns=["Flow","dev","t-4dev","t-8dev","t-12dev","t-16dev","t-20dev","t-24dev","t-28dev","t-32dev","t-36dev","t-40dev","t20Leak","t24Leak","t28Leak","t32Leak","t36Leak","t40Leak"]))

In [10]:
    model = Sequential()
    model.add(Dense(12, input_dim=12, kernel_initializer ='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(500, kernel_initializer ='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(500, kernel_initializer ='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer ='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    
model.fit(XTrain, yTrain, validation_data=(XTest,yTest), epochs=10, batch_size=20)

Train on 100000 samples, validate on 12520 samples
Epoch 1/10
100000/100000 [==============================] - 58s 579us/step - loss: 0.5982 - acc: 0.6914 - val_loss: 0.6020 - val_acc: 0.8723
Epoch 2/10
100000/100000 [==============================] - 58s 577us/step - loss: 0.5865 - acc: 0.7006 - val_loss: 0.4763 - val_acc: 0.8870
Epoch 3/10
100000/100000 [==============================] - 57s 574us/step - loss: 0.5796 - acc: 0.7076 - val_loss: 0.4839 - val_acc: 0.8777
Epoch 4/10
100000/100000 [==============================] - 57s 575us/step - loss: 0.5744 - acc: 0.7096 - val_loss: 0.4724 - val_acc: 0.8847
Epoch 5/10
100000/100000 [==============================] - 57s 574us/step - loss: 0.5704 - acc: 0.7130 - val_loss: 0.4712 - val_acc: 0.8840
Epoch 6/10
100000/100000 [==============================] - 57s 574us/step - loss: 0.5676 - acc: 0.7147 - val_loss: 0.4615 - val_acc: 0.8896
Epoch 7/10
100000/100000 [==============================] - 57s 575us/step - loss: 0.5661 - acc: 0.7166